<a href="https://colab.research.google.com/github/emerson-matos/tcc-notebooks/blob/main/tcc_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monta o google drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Dados dos hoteis

In [24]:
date = "2023/02/15"

## Cria o DataFrame

In [2]:
import pandas as pd

df = pd.read_csv("/content/gdrive/MyDrive/tcc/data/2023/02/15/aram-beach-&-convention-gm-reviews.csv")

df[:1]

,token,review_id,retrieval_date,rating,rating_max,relative_date,likes,other_ratings,trip_type_travel_group,user_name,user_is_local_guide,user_reviews,user_photos,user_url,text,response_text,response_relative_date,errors
0,CAESBkVnSUlDZw==,ChZDSUhNMG9nS0VJQ0FnSUNob1lud2ZBEAE,2023-02-15 02:48:02.460131,5.0,5.0,5 horas atrás,-1,"Destaques do hotel Luxuoso, Vista linda e Tran...",Negócios | Amigos,Lucas Victor da Luz Cesar,True,21,12.0,https://www.google.com/maps/contrib/1109357607...,NaN,NaN,NaN,[]


In [20]:
df.loc[df['text'].notnull()].shape[0]

1652

In [65]:
import pandas as pd
import glob
import os

path_to_lookup = f'/content/gdrive/MyDrive/tcc/data/{date}/'
file_pattern = "*.csv"

# setting the path for joining multiple files
files = os.path.join(path_to_lookup, file_pattern)

# list of merged files returned
files = glob.glob(files)

df = pd.concat(map(lambda file: pd.read_csv(file).assign(source=file.split("/")[-1]), files), ignore_index=True)
df.shape[0]
df = df.loc[df['text'].notna()]
# regfiles

In [69]:
df[['source', 'rating']].groupby(['source']).size().reset_index(name='counts').sort_values("counts", ascending=False)

,source,counts
52,pratagy-beach-all-inclusive-resort-maceió---w...,19777
14,grand-palladium-imbassaí-resort-&-spa-gm-revi...,15828
5,costão-do-santinho-resort-all-inclusive-gm-re...,10517
41,iberostar-bahia-gm-reviews.csv,10045
61,serhs-natal-grand-hotel-&-resort-gm-reviews.csv,8131
...,...,...
7,dan-inn-são-paulo-higienópolis-gm-reviews.csv,637
42,iberostar-grand-amazon-gm-reviews.csv,617
17,hotel-atlântico-avenida-gm-reviews.csv,404
24,hotel-beira-mar-gm-reviews.csv,384


In [70]:
df.describe()

,rating,rating_max,user_photos
count,209946.00000,209946.0,117705.000000
mean,4.39211,5.0,68.633708
std,1.10198,0.0,141.699008
min,1.00000,5.0,1.000000
25%,4.00000,5.0,3.000000
50%,5.00000,5.0,13.000000
75%,5.00000,5.0,58.000000
max,5.00000,5.0,999.000000
